In [1]:
import pandas as pd  # type: ignore

times_df = pd.read_csv('processed_data/ordered_times.csv')

# Display basic information about the DataFrame
print(f"Shape: {times_df.shape}")
print(f"Columns: {list(times_df.columns)}")
times_df.head()


Shape: (4173, 11)
Columns: ['SHEET', 'SEX', 'EVENT', ' TIME', 'NAME', 'YEAR', 'TEAM', 'RANK', 'SITE', 'MEET', 'CONTEXT']


,SHEET,SEX,EVENT,TIME,NAME,YEAR,TEAM,RANK,SITE,MEET,CONTEXT
0,CMS All Time Top 10,Athena,50 FREE,23.29,Jocelyn Crawford,2019,NaN,NaN,NaN,NaN,NaN
1,CMS All Time Top 10,Athena,50 FREE,23.31,Ava Sealander,2022,NaN,NaN,NaN,NaN,NaN
2,CMS All Time Top 10,Athena,50 FREE,23.39,Madeleine Kan,2025,NaN,NaN,NaN,NaN,NaN
3,CMS All Time Top 10,Athena,50 FREE,23.49,Kelly Ngo,2016,NaN,NaN,NaN,NaN,NaN
4,CMS All Time Top 10,Athena,50 FREE,23.71,Helen Liu,2014,NaN,NaN,NaN,NaN,NaN
